
# **Modeling Flu Spread in Bucharest using Monte Carlo Simulation** #


##### The aim of this project is to model the spread of the flu in Bucharest using the Monte Carlo method to understand the dynamics of a virus in a crowded city. #####

In [1]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

##### In this simulation model for Bucharest, we used a sample population of 10,000 individuals, with an estimated probability of infection between 3-5% per contact and an average recovery time of 7-10 days. The simulation is carried out over a 90-day period to cover a typical influenza season, allowing to observe the evolution of the spread of infection in a large city. ##### 

In [2]:
# Parameters for the simulation
nr_populatie = 10000
prob_infectare = np.random.uniform(0.03, 0.05, nr_populatie)  # Probability of infection per individual
nr_zile_refacere = np.zeros(nr_populatie, dtype=int)         
simulare_zile = 150                                           # Number of simulation days (1 nov - 31 mar)
nr_sectoare = 6
nr_simulari = 1000 

In [3]:
# Simulation of the spread of the virus in the population every day
nr_infectati_pe_zi = []

for zi in range(simulare_zile):
    posibil_infectati = nr_zile_refacere == 0  # Not currently recovering
    sansa_infectare = np.random.rand(nr_populatie) 
    infectati = posibil_infectati & (sansa_infectare < prob_infectare)  # If random number is less than infection probability
    nr_infectati_pe_zi.append(np.sum(infectati))

    # Set recovery days for newly infected individuals
    nr_zile_refacere[infectati] = np.random.randint(7, 11, size=np.sum(infectati))

    # Decrease recovery days for all those recovering
    nr_zile_refacere[nr_zile_refacere > 0] -= 1

# Display results
print("Nr infectati pe zi: ", nr_infectati_pe_zi)



# Probability Estimation Using Monte Carlo Method
infected_count = 0  # Count of favorable outcomes

# Run trials for estimating infection probability
for _ in range(nr_simulari):
    # Simulate each individual's infection status
    for i in range(nr_populatie):
        random_value = np.random.rand()  # Generate a random number for this individual
        if random_value < prob_infectare[i]:  # Compare with individual infection probability
            infected_count += 1  # Increment count if infected

# Estimate probability
estimated_probability = infected_count / (nr_simulari * nr_populatie)  
print(f"Estimated Probability of Infection: {estimated_probability:.4f}")

Nr infectati pe zi:  [404, 402, 380, 340, 361, 344, 338, 297, 326, 278, 309, 300, 288, 318, 297, 286, 306, 331, 295, 302, 302, 311, 271, 315, 321, 334, 339, 258, 312, 292, 292, 298, 309, 290, 331, 284, 306, 288, 291, 325, 303, 291, 299, 295, 299, 292, 300, 300, 337, 311, 289, 301, 319, 323, 282, 330, 319, 328, 291, 284, 292, 299, 306, 341, 312, 300, 338, 295, 299, 309, 315, 287, 297, 308, 337, 292, 290, 288, 310, 289, 309, 336, 318, 286, 301, 298, 305, 283, 287, 321, 308, 306, 304, 318, 288, 297, 309, 292, 299, 276, 292, 297, 334, 310, 319, 284, 309, 338, 322, 278, 290, 302, 297, 292, 317, 326, 311, 323, 296, 287, 286, 306, 293, 312, 313, 304, 331, 296, 311, 296, 294, 305, 321, 302, 298, 325, 306, 289, 286, 304, 270, 311, 302, 323, 310, 302, 304, 290, 293, 306]
Estimated Probability of Infection: 0.0399


In [4]:
# Plotting the evolution of infected numbers each day
zile_pe_luna = [30, 31, 31, 28, 31]
luni = ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'] 
pozitie_luna = [i * zile_pe_luna[i] for i in range(len(zile_pe_luna))]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(simulare_zile), 
    y=nr_infectati_pe_zi, 
    mode='lines', 
    name='Numar infectati in perioda noiembrie - martie', 
    line=dict(color='MediumPurple', width=2)))

fig.update_layout(title=dict(text='Evolutia gripei in Bucuresti (noiembrie - martie)', x=0.5),
                   xaxis=dict(title='Zile', tickvals=pozitie_luna, ticktext=luni),
                   yaxis_title='Nr persoane infectate',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font=dict(family='system-ui', size=14, color='white'),
                   plot_bgcolor='Gainsboro', )
fig.show()

In [5]:
# Simulation parameters per sector
populatie_simulata_sectoare = np.array([1192, 1745, 2244, 1546, 1422, 1849])  # Simulated population for each sector
prob_infectare_sectoare = np.random.uniform(0.03, 0.05, nr_sectoare)          # Infection probability per sector

# Initialize recovery arrays for each sector
nr_zile_refacere_sectoare = [np.zeros(pop, dtype=int) for pop in populatie_simulata_sectoare]

# List to hold the number of infected individuals per day for each sector
nr_infectati_pe_zi_sectoare = [[] for _ in range(nr_sectoare)]

for zi in range(simulare_zile):
    for s in range(nr_sectoare):
        # Determine those who are susceptible to infection (not recovering)
        posibil_infectati = nr_zile_refacere_sectoare[s] == 0
        sansa_infectare = np.random.rand(populatie_simulata_sectoare[s])

        # Determine new infections
        infectati = posibil_infectati & (sansa_infectare < prob_infectare_sectoare[s])
        nr_infectati_pe_zi_sectoare[s].append(np.sum(infectati))

        # Set recovery days for newly infected individuals
        nr_zile_refacere_sectoare[s][infectati] = np.random.randint(7, 11, size=np.sum(infectati))

        # Decrement recovery days for those who are still in recovery
        nr_zile_refacere_sectoare[s][nr_zile_refacere_sectoare[s] > 0] -= 1

# Display results for each district
for s in range(nr_sectoare):
    print(f"Nr infectati pe zi in sectorul {s+1}: {nr_infectati_pe_zi_sectoare[s]}")


Nr infectati pe zi in sectorul 1: [59, 53, 42, 48, 56, 41, 41, 54, 43, 36, 41, 33, 40, 36, 52, 45, 39, 39, 43, 48, 52, 36, 45, 47, 44, 47, 40, 40, 40, 50, 36, 45, 48, 46, 44, 35, 39, 38, 49, 33, 40, 55, 51, 41, 55, 44, 41, 45, 36, 35, 45, 32, 34, 40, 36, 50, 54, 51, 33, 47, 57, 35, 38, 41, 37, 51, 38, 39, 45, 51, 48, 37, 49, 45, 38, 53, 44, 43, 46, 42, 41, 42, 38, 34, 46, 40, 44, 42, 51, 44, 28, 40, 35, 61, 45, 45, 49, 37, 38, 47, 39, 31, 40, 42, 40, 41, 32, 50, 33, 43, 33, 48, 55, 34, 31, 46, 41, 44, 47, 51, 49, 40, 49, 37, 39, 35, 39, 45, 34, 50, 47, 56, 50, 37, 35, 47, 39, 38, 45, 42, 41, 37, 40, 41, 46, 38, 38, 50, 41, 37]
Nr infectati pe zi in sectorul 2: [67, 75, 79, 61, 56, 55, 44, 61, 48, 52, 56, 54, 61, 67, 54, 60, 47, 55, 53, 58, 39, 51, 51, 55, 55, 64, 51, 57, 44, 55, 47, 64, 41, 53, 59, 58, 62, 63, 57, 57, 54, 57, 46, 45, 55, 61, 55, 52, 69, 42, 54, 64, 54, 66, 55, 54, 58, 53, 52, 42, 67, 64, 56, 69, 49, 39, 45, 46, 49, 52, 51, 50, 59, 49, 54, 62, 60, 58, 45, 68, 48, 49, 40

In [6]:
# Calculate total infections for each sector
infectii_pe_sector_toate_zilele = [sum(nr_infectati_pe_zi_sectoare[s]) for s in range(nr_sectoare)]

# Calculate infection rates for each sector
infectie_rate_sectoare = [infectii / populatie_simulata_sectoare[s] for s, infectii in enumerate(infectii_pe_sector_toate_zilele)] 

# Define sector labels that match the GeoJSON
idx_sector = ['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6']

# Create a DataFrame with the sector labels and infection rates
data_rate = pd.DataFrame({'Sector': idx_sector, 'Infection Rate': infectie_rate_sectoare})
print(data_rate)

# Load the GeoJSON file for Bucharest sectors
geojson_fisier = '/Users/alexandrazamfirescu/Bucharest_Streets/Flu_Spread_Simulation/BucurestiSectoare.geo.json'

with open(geojson_fisier) as f:
    geojson = json.load(f)

# Plot the choropleth map for infection rates
fig = px.choropleth(
    data_frame=data_rate,
    geojson=geojson_fisier,
    locations='Sector',
    featureidkey='properties.tags.name',  # Ensure this matches your GeoJSON
    color='Infection Rate',
    color_continuous_scale='Turbo',
    labels={'Infection Rate': 'Infection Rate'},
    projection='mercator'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title=dict(text='Rata de infectare cu gripa in fiecare sector', x=0.5),
    margin={"r":0, "t":40, "l":0, "b":0}
)
fig.show()


     Sector  Infection Rate
0  Sector 1        5.381711
1  Sector 2        4.744413
2  Sector 3        4.027184
3  Sector 4        4.033635
4  Sector 5        3.700422
5  Sector 6        4.343970
